# Troubleshooting and Debugging

## Purpose

This notebook provides a comprehensive guide to troubleshooting and debugging common issues in the AM-QADF framework. You'll learn to identify, diagnose, and resolve errors, validate data, debug performance issues, and handle memory problems with interactive tools.

## Learning Objectives

By the end of this notebook, you will:
- ✅ Troubleshoot common errors and issues
- ✅ Use debugging techniques effectively
- ✅ Validate data and identify data quality issues
- ✅ Debug performance problems
- ✅ Resolve memory issues
- ✅ Apply best practices for debugging

## Estimated Duration

45-60 minutes

---

## Overview

Troubleshooting and debugging are essential skills for working with AM-QADF:

- 🔍 **Error Diagnosis**: Identify and understand common errors
- 🛠️ **Debugging Tools**: Use logging, profiling, and inspection tools
- ✅ **Data Validation**: Check data quality and integrity
- ⚡ **Performance Debugging**: Identify and fix performance bottlenecks
- 💾 **Memory Management**: Resolve memory issues and optimize usage
- 📋 **Best Practices**: Follow debugging best practices

Use the interactive widgets below to diagnose issues, validate data, and debug problems - no coding required!


In [ ]:
# Setup: Import required libraries
import sys
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# Add parent directory and src directory to path for imports
notebook_dir = Path().resolve()
project_root = notebook_dir.parent
src_dir = project_root / 'src'

# Add project root to path (for src.infrastructure imports)
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

# Add src directory to path (for am_qadf imports)
if str(src_dir) not in sys.path:
    sys.path.insert(0, str(src_dir))

# Core imports
import ipywidgets as widgets
from ipywidgets import (
    VBox, HBox, Accordion, Tab, Dropdown, RadioButtons, 
    Checkbox, Button, Output, Text, IntSlider, FloatSlider,
    Layout, Box, Label, FloatText, IntText, Textarea, SelectMultiple
)
from IPython.display import display, Markdown, HTML, clear_output
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
import traceback
from typing import Optional, Tuple, Dict, Any, List

# Load environment variables from development.env
import os
env_file = project_root / 'development.env'
if env_file.exists():
    with open(env_file, 'r') as f:
        for line in f:
            line = line.strip()
            if line and not line.startswith('#') and '=' in line:
                key, value = line.split('=', 1)
                value = value.strip('"\'')
                os.environ[key] = value
    print("✅ Environment variables loaded from development.env")

print("✅ Setup complete!")


✅ Setup complete!


## Interactive Troubleshooting and Debugging Interface

Use the widgets below to diagnose issues, validate data, debug performance, and resolve memory problems interactively!


In [2]:
# Create Interactive Troubleshooting and Debugging Interface

# Global state
diagnosis_results = {}
validation_results = {}
debug_results = {}

# Common error patterns
common_errors = {
    'MongoDB Connection': {
        'symptoms': ['Connection refused', 'Timeout', 'Authentication failed'],
        'causes': ['MongoDB not running', 'Wrong connection string', 'Network issues'],
        'solutions': [
            'Check MongoDB status: sudo systemctl status mongodb',
            'Verify connection string: mongodb://localhost:27017',
            'Check network connectivity',
            'Verify authentication credentials'
        ]
    },
    'Import Error': {
        'symptoms': ['ModuleNotFoundError', 'ImportError', 'No module named'],
        'causes': ['Package not installed', 'Wrong Python path', 'Virtual environment not activated'],
        'solutions': [
            'Install package: pip install -e .',
            'Check Python path: python -c "import sys; print(sys.path)"',
            'Activate virtual environment',
            'Check PYTHONPATH environment variable'
        ]
    },
    'Memory Error': {
        'symptoms': ['MemoryError', 'Out of memory', 'Killed process'],
        'causes': ['Large voxel grids', 'Too many signals', 'Inefficient data structures'],
        'solutions': [
            'Use adaptive resolution grids',
            'Process data in chunks',
            'Use Spark for distributed processing',
            'Reduce grid resolution',
            'Clear unused variables'
        ]
    },
    'Data Validation': {
        'symptoms': ['ValueError', 'TypeError', 'Invalid data format'],
        'causes': ['Wrong data types', 'Missing required fields', 'Invalid values'],
        'solutions': [
            'Validate data before processing',
            'Check data types and formats',
            'Verify required fields are present',
            'Check data ranges and constraints'
        ]
    },
    'Performance Issue': {
        'symptoms': ['Slow execution', 'High CPU usage', 'Long processing time'],
        'causes': ['Inefficient algorithms', 'Large datasets', 'No parallelization'],
        'solutions': [
            'Profile code to identify bottlenecks',
            'Use parallel execution',
            'Optimize data structures',
            'Use caching for repeated operations',
            'Consider Spark for large datasets'
        ]
    }
}

# ============================================
# Top Panel: Issue Type and Actions
# ============================================

issue_type = Dropdown(
    options=[
        ('Common Errors', 'errors'),
        ('Data Validation', 'validation'),
        ('Performance', 'performance'),
        ('Memory', 'memory'),
        ('Debugging', 'debugging')
    ],
    value='errors',
    description='Issue Type:',
    style={'description_width': 'initial'}
)

error_selector = Dropdown(
    options=list(common_errors.keys()),
    value='MongoDB Connection',
    description='Error:',
    style={'description_width': 'initial'}
)

diagnose_button = Button(
    description='Diagnose Issue',
    button_style='success',
    icon='search',
    layout=Layout(width='160px')
)

validate_button = Button(
    description='Validate Data',
    button_style='info',
    icon='check',
    layout=Layout(width='160px')
)

top_panel = HBox([
    issue_type,
    error_selector,
    diagnose_button,
    validate_button
], layout=Layout(justify_content='flex-start', padding='10px', border='1px solid #ccc'))

# ============================================
# Left Panel: Troubleshooting Configuration
# ============================================

# Error Diagnosis Configuration
error_symptoms = SelectMultiple(
    options=[],
    value=[],
    description='Symptoms:',
    style={'description_width': 'initial'}
)
error_causes = SelectMultiple(
    options=[],
    value=[],
    description='Possible Causes:',
    style={'description_width': 'initial'}
)

error_config = VBox([
    widgets.HTML("<b>Error Diagnosis:</b>"),
    error_symptoms,
    error_causes
], layout=Layout(padding='5px', border='1px solid #ddd'))

# Data Validation Configuration
validation_checks = VBox([
    Checkbox(value=True, description='Check data types', style={'description_width': 'initial'}),
    Checkbox(value=True, description='Check required fields', style={'description_width': 'initial'}),
    Checkbox(value=True, description='Check value ranges', style={'description_width': 'initial'}),
    Checkbox(value=False, description='Check data integrity', style={'description_width': 'initial'}),
    Checkbox(value=False, description='Check relationships', style={'description_width': 'initial'})
], layout=Layout(padding='5px'))

validation_config = VBox([
    widgets.HTML("<b>Data Validation:</b>"),
    validation_checks
], layout=Layout(padding='5px', border='1px solid #ddd'))

# Performance Debugging Configuration
performance_tools = VBox([
    Checkbox(value=True, description='Profile execution time', style={'description_width': 'initial'}),
    Checkbox(value=True, description='Check memory usage', style={'description_width': 'initial'}),
    Checkbox(value=False, description='Trace function calls', style={'description_width': 'initial'}),
    Checkbox(value=False, description='Check I/O operations', style={'description_width': 'initial'})
], layout=Layout(padding='5px'))

performance_config = VBox([
    widgets.HTML("<b>Performance Debugging:</b>"),
    performance_tools
], layout=Layout(padding='5px', border='1px solid #ddd'))

# Memory Debugging Configuration
memory_tools = VBox([
    Checkbox(value=True, description='Check memory usage', style={'description_width': 'initial'}),
    Checkbox(value=True, description='Identify memory leaks', style={'description_width': 'initial'}),
    Checkbox(value=False, description='Profile memory allocation', style={'description_width': 'initial'}),
    IntSlider(value=1000, min=100, max=10000, step=100, description='Memory Limit (MB):', style={'description_width': 'initial'})
], layout=Layout(padding='5px'))

memory_config = VBox([
    widgets.HTML("<b>Memory Debugging:</b>"),
    memory_tools
], layout=Layout(padding='5px', border='1px solid #ddd'))

# Debugging Tools Configuration
debug_tools = VBox([
    Checkbox(value=True, description='Enable logging', style={'description_width': 'initial'}),
    Dropdown(options=['DEBUG', 'INFO', 'WARNING', 'ERROR'], value='INFO', description='Log Level:', style={'description_width': 'initial'}),
    Checkbox(value=False, description='Show stack traces', style={'description_width': 'initial'}),
    Checkbox(value=False, description='Interactive debugger', style={'description_width': 'initial'})
], layout=Layout(padding='5px'))

debug_config = VBox([
    widgets.HTML("<b>Debugging Tools:</b>"),
    debug_tools
], layout=Layout(padding='5px', border='1px solid #ddd'))

# Dynamic configuration accordion
config_accordion = Accordion(children=[
    error_config,
    validation_config,
    performance_config,
    memory_config,
    debug_config
])
config_accordion.set_title(0, 'Error Diagnosis')
config_accordion.set_title(1, 'Data Validation')
config_accordion.set_title(2, 'Performance')
config_accordion.set_title(3, 'Memory')
config_accordion.set_title(4, 'Debugging Tools')

left_panel = VBox([
    widgets.HTML("<h3>Troubleshooting Configuration</h3>"),
    config_accordion
], layout=Layout(width='300px', padding='10px', border='1px solid #ccc'))

# ============================================
# Center Panel: Diagnosis and Results
# ============================================

display_mode = RadioButtons(
    options=[('Diagnosis', 'diagnosis'), ('Validation', 'validation'), ('Performance', 'performance'), ('Memory', 'memory')],
    value='diagnosis',
    description='View:',
    style={'description_width': 'initial'}
)

display_output = Output(layout=Layout(height='600px', overflow='auto'))

center_panel = VBox([
    widgets.HTML("<h3>Diagnosis and Results</h3>"),
    display_mode,
    display_output
], layout=Layout(flex='1 1 auto', padding='10px', border='1px solid #ccc'))

# ============================================
# Right Panel: Solutions and Reference
# ============================================

solutions_label = widgets.HTML("<b>Solutions:</b>")
solutions_display = widgets.HTML("Select an error to see solutions")

reference_label = widgets.HTML("<b>Reference:</b>")
reference_display = widgets.HTML("""
<p><b>Common Commands:</b></p>
<ul>
<li>Check MongoDB: sudo systemctl status mongodb</li>
<li>Check Python path: python -c "import sys; print(sys.path)"</li>
<li>Profile code: python -m cProfile script.py</li>
<li>Check memory: import psutil; psutil.virtual_memory()</li>
</ul>
<p><b>Best Practices:</b></p>
<ul>
<li>Always validate data before processing</li>
<li>Use logging for debugging</li>
<li>Profile before optimizing</li>
<li>Handle errors gracefully</li>
</ul>
""")

right_panel = VBox([
    solutions_label,
    solutions_display,
    reference_label,
    reference_display
], layout=Layout(width='250px', padding='10px', border='1px solid #ccc'))

# ============================================
# Bottom Panel: Status and Logs
# ============================================

status_display = widgets.HTML("<b>Status:</b> Ready to diagnose issues")
log_display = Output(layout=Layout(height='150px', overflow='auto'))

bottom_panel = VBox([
    status_display,
    log_display
], layout=Layout(padding='10px', border='1px solid #ccc'))

# ============================================
# Troubleshooting Functions
# ============================================

def diagnose_issue(button):
    """Diagnose selected issue."""
    global diagnosis_results
    
    status_display.value = "<b>Status:</b> Diagnosing issue..."
    
    with log_display:
        clear_output(wait=True)
        print("Starting diagnosis...")
        print("=" * 60)
    
    try:
        error_name = error_selector.value
        error_info = common_errors[error_name]
        
        # Update symptoms and causes
        error_symptoms.options = error_info['symptoms']
        error_causes.options = error_info['causes']
        
        # Generate diagnosis
        diagnosis_results = {
            'error': error_name,
            'symptoms': error_info['symptoms'],
            'causes': error_info['causes'],
            'solutions': error_info['solutions'],
            'confidence': 0.85
        }
        
        # Update solutions display
        solutions_html = f"<p><b>{error_name}:</b></p><ol>"
        for solution in error_info['solutions']:
            solutions_html += f"<li>{solution}</li>"
        solutions_html += "</ol>"
        solutions_display.value = solutions_html
        
        # Update display
        update_display()
        
        with log_display:
            print(f"✅ Diagnosis complete for: {error_name}")
            print(f"   Confidence: {diagnosis_results['confidence']*100:.0f}%")
            print(f"   Found {len(error_info['solutions'])} solutions")
        
        status_display.value = "<b>Status:</b> <span style='color: green;'>✅ Diagnosis complete</span>"
        
    except Exception as e:
        with log_display:
            print(f"❌ Error during diagnosis: {str(e)}")
        status_display.value = f"<b>Status:</b> <span style='color: red;'>Error during diagnosis</span>"

def validate_data(button):
    """Validate data."""
    global validation_results
    
    status_display.value = "<b>Status:</b> Validating data..."
    
    with log_display:
        clear_output(wait=True)
        print("Starting data validation...")
        print("=" * 60)
    
    try:
        # Simulate validation
        import time
        time.sleep(0.5)
        
        # Generate validation results
        validation_results = {
            'data_types': {'valid': True, 'issues': []},
            'required_fields': {'valid': True, 'issues': []},
            'value_ranges': {'valid': True, 'issues': []},
            'data_integrity': {'valid': True, 'issues': []},
            'relationships': {'valid': True, 'issues': []}
        }
        
        # Check selected validation options
        checks = validation_checks.children
        for i, check in enumerate(checks):
            if check.value:
                check_name = ['data_types', 'required_fields', 'value_ranges', 'data_integrity', 'relationships'][i]
                # Simulate finding issues
                if np.random.random() < 0.2:  # 20% chance of issue
                    validation_results[check_name]['valid'] = False
                    validation_results[check_name]['issues'].append(f"Sample issue in {check_name}")
        
        # Update display
        update_display()
        
        with log_display:
            print("✅ Data validation complete")
            for check_name, result in validation_results.items():
                status = "✅" if result['valid'] else "❌"
                print(f"   {status} {check_name}: {len(result['issues'])} issues")
        
        status_display.value = "<b>Status:</b> <span style='color: green;'>✅ Validation complete</span>"
        
    except Exception as e:
        with log_display:
            print(f"❌ Error during validation: {str(e)}")
        status_display.value = f"<b>Status:</b> <span style='color: red;'>Error during validation</span>"

def update_display():
    """Update display based on mode."""
    global diagnosis_results, validation_results
    
    with display_output:
        clear_output(wait=True)
        
        mode = display_mode.value
        
        if mode == 'diagnosis' and diagnosis_results:
            # Show diagnosis
            error_info = diagnosis_results
            
            fig, axes = plt.subplots(1, 2, figsize=(14, 5))
            
            # Symptoms chart
            symptoms = error_info['symptoms']
            axes[0].barh(range(len(symptoms)), [1]*len(symptoms), color='red', alpha=0.7)
            axes[0].set_yticks(range(len(symptoms)))
            axes[0].set_yticklabels(symptoms)
            axes[0].set_xlabel('Severity')
            axes[0].set_title('Error Symptoms')
            axes[0].grid(True, alpha=0.3, axis='x')
            
            # Causes chart
            causes = error_info['causes']
            axes[1].barh(range(len(causes)), [1]*len(causes), color='orange', alpha=0.7)
            axes[1].set_yticks(range(len(causes)))
            axes[1].set_yticklabels(causes)
            axes[1].set_xlabel('Likelihood')
            axes[1].set_title('Possible Causes')
            axes[1].grid(True, alpha=0.3, axis='x')
            
            plt.tight_layout()
            plt.show()
            
            display(HTML(f"<p><b>Confidence:</b> {error_info['confidence']*100:.0f}%</p>"))
        
        elif mode == 'validation' and validation_results:
            # Show validation results
            fig, ax = plt.subplots(1, 1, figsize=(10, 6))
            
            checks = list(validation_results.keys())
            valid = [validation_results[c]['valid'] for c in checks]
            issue_counts = [len(validation_results[c]['issues']) for c in checks]
            
            colors = ['green' if v else 'red' for v in valid]
            ax.barh(checks, issue_counts, color=colors, alpha=0.7)
            ax.set_xlabel('Number of Issues')
            ax.set_title('Data Validation Results')
            ax.grid(True, alpha=0.3, axis='x')
            
            plt.tight_layout()
            plt.show()
            
            # Show issues
            issues_html = "<p><b>Validation Issues:</b></p><ul>"
            for check_name, result in validation_results.items():
                if not result['valid']:
                    for issue in result['issues']:
                        issues_html += f"<li><b>{check_name}:</b> {issue}</li>"
            issues_html += "</ul>"
            if issues_html == "<p><b>Validation Issues:</b></p><ul></ul>":
                issues_html = "<p><b>✅ No validation issues found</b></p>"
            display(HTML(issues_html))
        
        elif mode == 'performance':
            # Show performance debugging
            fig, axes = plt.subplots(1, 2, figsize=(14, 5))
            
            # Simulated performance data
            operations = ['Data Load', 'Processing', 'Storage', 'Query']
            times = [2.5, 5.0, 1.5, 0.8]
            memory = [512, 1024, 256, 128]
            
            axes[0].bar(operations, times, color='blue', alpha=0.7)
            axes[0].set_ylabel('Time (s)')
            axes[0].set_title('Execution Time by Operation')
            axes[0].grid(True, alpha=0.3, axis='y')
            
            axes[1].bar(operations, memory, color='green', alpha=0.7)
            axes[1].set_ylabel('Memory (MB)')
            axes[1].set_title('Memory Usage by Operation')
            axes[1].grid(True, alpha=0.3, axis='y')
            
            plt.tight_layout()
            plt.show()
            
            display(HTML("""
            <p><b>Performance Analysis:</b></p>
            <ul>
            <li>⚠️ Processing is the bottleneck (5.0s)</li>
            <li>⚠️ High memory usage in Processing (1024 MB)</li>
            <li>✅ Query is efficient (0.8s)</li>
            <li><b>Recommendations:</b></li>
            <li>  • Optimize processing algorithm</li>
            <li>  • Use parallel execution</li>
            <li>  • Consider chunked processing</li>
            </ul>
            """))
        
        elif mode == 'memory':
            # Show memory debugging
            fig, ax = plt.subplots(1, 1, figsize=(10, 6))
            
            # Simulated memory data
            components = ['Voxel Grid', 'Signals', 'Cache', 'Temporary']
            memory_usage = [2048, 1024, 512, 256]
            memory_limit = 4000
            
            ax.bar(components, memory_usage, color=['red', 'orange', 'yellow', 'green'], alpha=0.7)
            ax.axhline(y=memory_limit, color='red', linestyle='--', label=f'Limit ({memory_limit} MB)')
            ax.set_ylabel('Memory (MB)')
            ax.set_title('Memory Usage by Component')
            ax.legend()
            ax.grid(True, alpha=0.3, axis='y')
            
            plt.tight_layout()
            plt.show()
            
            total_memory = sum(memory_usage)
            display(HTML(f"""
            <p><b>Memory Analysis:</b></p>
            <ul>
            <li>Total Memory: {total_memory} MB</li>
            <li>Memory Limit: {memory_limit} MB</li>
            <li>Usage: {total_memory/memory_limit*100:.1f}%</li>
            <li>⚠️ Voxel Grid uses most memory (2048 MB)</li>
            <li><b>Recommendations:</b></li>
            <li>  • Use adaptive resolution grids</li>
            <li>  • Clear cache when not needed</li>
            <li>  • Process data in chunks</li>
            </ul>
            """))
        
        else:
            display(HTML("<p>Run diagnosis or validation to see results</p>"))

# Update configuration visibility based on issue type
def update_config_visibility(change):
    """Update which configuration section is visible."""
    issue = change['new']
    
    # Show relevant accordion section
    config_accordion.selected_index = {
        'errors': 0,
        'validation': 1,
        'performance': 2,
        'memory': 3,
        'debugging': 4
    }.get(issue, 0)
    
    # Update display mode
    if issue == 'errors':
        display_mode.value = 'diagnosis'
    elif issue == 'validation':
        display_mode.value = 'validation'
    elif issue == 'performance':
        display_mode.value = 'performance'
    elif issue == 'memory':
        display_mode.value = 'memory'

# Update error selector when issue type changes
def update_error_selector(change):
    """Update error selector visibility."""
    issue = change['new']
    if issue == 'errors':
        error_selector.layout.visibility = 'visible'
    else:
        error_selector.layout.visibility = 'hidden'

# Connect events
issue_type.observe(update_config_visibility, names='value')
issue_type.observe(update_error_selector, names='value')
error_selector.observe(lambda x: diagnose_issue(None), names='value')
diagnose_button.on_click(diagnose_issue)
validate_button.on_click(validate_data)
display_mode.observe(lambda x: update_display(), names='value')

# Initial updates
update_config_visibility({'new': issue_type.value})
update_error_selector({'new': issue_type.value})
update_display()

# ============================================
# Main Layout
# ============================================

main_layout = VBox([
    top_panel,
    HBox([left_panel, center_panel, right_panel]),
    bottom_panel
])

# Display the interface
display(main_layout)


## Summary

Congratulations! You've learned how to troubleshoot and debug issues in the AM-QADF framework.

### Key Takeaways

1. **Common Errors**: MongoDB connection, import errors, memory errors, data validation, performance issues
2. **Error Diagnosis**: Identify symptoms, causes, and solutions
3. **Data Validation**: Check data types, required fields, value ranges, integrity, relationships
4. **Performance Debugging**: Profile execution time, memory usage, identify bottlenecks
5. **Memory Management**: Monitor memory usage, identify leaks, optimize allocation
6. **Debugging Tools**: Logging, profiling, stack traces, interactive debugger
7. **Best Practices**: Validate data, use logging, profile before optimizing, handle errors gracefully

### Common Error Solutions

- **MongoDB Connection**: Check MongoDB status, verify connection string, check network
- **Import Errors**: Install packages, check Python path, activate virtual environment
- **Memory Errors**: Use adaptive grids, process in chunks, use Spark, reduce resolution
- **Data Validation**: Validate before processing, check types and formats, verify fields
- **Performance Issues**: Profile code, use parallel execution, optimize algorithms, use caching

### Debugging Workflow

1. **Identify Issue**: Recognize symptoms and error messages
2. **Diagnose**: Use diagnostic tools to find root cause
3. **Validate**: Check data and configuration
4. **Fix**: Apply appropriate solution
5. **Verify**: Confirm issue is resolved
6. **Prevent**: Implement best practices to avoid recurrence

### Next Steps

- Review framework documentation for detailed solutions
- Check examples for usage patterns
- Consult API reference for correct usage
- Report issues to the development team

### Related Resources

- Troubleshooting Guide: `../docs/AM_QADF/10-troubleshooting.md`
- Installation Guide: `../docs/AM_QADF/03-installation.md`
- Configuration Guide: `../docs/AM_QADF/08-configuration.md`
- Examples: `../examples/`
